# Simulation Eval

In [1]:
%reload_ext autoreload
%autoreload 2
import os
import numpy as np
import torch
from pathlib import Path
from cvla.hf_model_class import cVLA_wrapped
from cvla.data_loader_h5 import H5Dataset
from cvla.data_loader_paired import PairedDataset
from cvla.utils_eval import Evaluator
from cvla.utils_trajectory import DummyCamera

from IPython.display import display, HTML
from cvla.utils_vis import render_example

os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/home/zhangj/miniconda3/envs/mani_skill_clean/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/zhangj/miniconda3/envs/mani_skill_clean/lib/python3.12/site-packages/sapien/_vulkan_tricks.py:37: UserWarning: Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.
  warn("Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.")


In [3]:
import getpass
'''
username = getpass.getuser()

if username == "bratulic":
    model_root = Path("/work/dlclarge2/bratulic-cvla/models/clevr-act-7-depth_img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_nexEnc_RealEval_baseline_augs_2025-04-11_15-52-11")
    model_path = model_root / "checkpoint-1750"
    model_root2 = Path("/work/dlclarge2/bratulic-cvla/models/clevr-act-7-depth_img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_NexEnc_augs_2025-04-10_16-32-05")
    model_path2 = model_root2 / "checkpoint-1200"
    model_root3 = Path("/work/dlclarge2/bratulic-cvla/models/clevr-act-7-depth_img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_nexEnc_RealEval_baseline_pc25_psim25_CameraPos_augs_2025-04-11_16-03-29")
    model_path3 = model_root3 / "checkpoint-2000"

    v14 = "cvla-clevr-camRF-sceneR-9__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_samplingAll_augs_2025-04-24_16-48-19" # 1 - 11
    #v15 = "cvla-clevr-camRF-sceneR-9__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_samplingAll_augs_warmup5_depthBoth_2025-04-25_16-46-38" # 1 - 11
    #v16 = "cvla-clevr-camRF-sceneR-9__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_samplingAll_augs_warmup5_depthOne_2025-04-25_07-51-29" # 8 - 22
    v17 = "cvla-clevr-camRF-sceneR-9__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_sortAll_augs_max20k_2025-04-27_18-00-52" # 6 - 20

    #v18 = "cvla-clevr-8__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_samplingAll_augs_warmup5_depth_2025-04-25_16-50-26" # 1k - 9k
    v19 = "cvla-clevr-8__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_samplingAll_augs_warmup5_depthBoth_2025-04-25_16-49-38" # 1k - 7k
    v20 = "cvla-clevr-8__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_sortAll_augs_max20k_2025-04-27_21-42-41" # 16 - 20
    v21 = "cvla-clevr-8__img_1_pr_interleaved_enc_xyzrotvec-cam-512xy128dmaxTokens13_lr3e-05_samplingAll_w05_maxSteps20k_2025-04-27_09-38-37" # 1 - 3

    v_depth = "mix30obj-camRF-sceneR-9_text_lr3e-05_mixObj_depth_enc128d_2025-04-29_22-05-39"
    v_text__v9 = "mix30obj-camRF-sceneR-9_text_lr3e-05_enc512_128d_normal_mix_2025-04-29_22-04-46"

    v9_no_augs = "cvla-clevr-camRF-sceneR-9_text_lr3e-05_clean_enc128d_noAugs_2025-04-29_22-09-11"
    mix_augs = "mix30obj-camRF-sceneR-9_text_lr3e-05_enc512_128d_normal_mix_2025-04-29_22-04-46"
    mix_depth_augs = "mix30obj-camRF-sceneR-9_text_lr3e-05_mixObj_depth_enc128d_2025-04-29_22-05-39"
    mix_no_augs = "mix30obj-camRF-sceneR-9_text_lr3e-05_mixObj_enc128d_noAugs_2025-04-29_22-07-13"
    v9_augs = "_text_lr3e-05_enc512_128d_normal_2025-04-29_19-18-16"
    v9_depth_augs = "_text_lr3e-05_enc512_128d_depth_2025-04-29_10-38-15"

    #model_path = Path("/work/dlclarge2/bratulic-cvla/models/") / v9_no_augs / "checkpoint-4687"
    model_path = Path("/work/dlclarge2/bratulic-cvla/models/") / v20 / "checkpoint-16000"
    dataset_for_simulation = "norm"

elif username == "argusm":
    model_location = Path("/data/lmbraid19/argusm/models/")
    #run = "_text_lr3e-05xyzrotvec-cam-1024xy_2025-04-23_11-04-12"
    #run = "_text_lr3e-05xyzrotvec-cam-256xy_2025-04-23_11-04-12"
    #run = "_text_lr3e-05xyzrotvec-cam-128xy_2025-04-23_11-04-12"
    run = "_text_lr3e-05xyzrotvec-cam-512xy256d_2025-04-23_12-03-48"
    #run = "_text_lr3e-05xyzrotvec-cam-512xy_2025-04-23_11-04-16"
    #run = "_text_lr3e-05xyzrotvec-rbt-256_2025-04-27_15-55-37"
    #run = "_text_lr3e-05_enc512_128d_depth_2025-04-29_10-38-15"
    #run = "_text_lr3e-05_enc512_256d_depth_2025-04-29_10-38-26"
    model_path = model_location / run / "checkpoint-4687"
'''
model_path = Path("/work/dlclarge2/zhangj-zhangj-CFM/models/training2_topview_70000_baseline/final_checkpoint")
model_wrapped = cVLA_wrapped(model_path=model_path )

Using device: cuda
loaded processor.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
conditioning = model_wrapped.conditioning
print(conditioning)

if conditioning == "trajectory":
    # simulation clevr dataset for testing
    
    if "cvla-clevr-8" in str(model_path):
        if dataset_for_simulation == "objaverse":
            dataset_location = Path("/tmp/cvla-obja-8")
        else:
            dataset_location = Path("/tmp/cvla-clevr-8-valid")
    else:
        if dataset_for_simulation == "objaverse":
            dataset_location = Path("/tmp/cvla-obja-camRF-sceneR-9")
        else:
            dataset_location = Path("/tmp/cvla-clevr-camRF-sceneR-9-valid")
        
    num_images_in_context = 1
    image_order = "interleaved"
    action_encoder = model_wrapped.enc_model.NAME
    raw_dataset = H5Dataset(dataset_location, action_encoder=action_encoder)
    run_name = f"_img_{num_images_in_context}_pr_{image_order}_enc_{action_encoder}"
    load_presampled_pairs_path = Path("/data/lmbraid21/bratulic/max_pali/datasets") / f"{dataset_location.name}_{run_name}_new.pkl"
    presampled_eval_sequences_path = Path("/data/lmbraid21/bratulic/max_pali/datasets") / f"{dataset_location.name}_{run_name}_pCopy0_pSorting0_presampled_eval_sequences.pkl"
    demonstration_dataset = PairedDataset(raw_dataset, num_images_in_context=num_images_in_context, image_order=image_order, load_presampled_pairs_path=load_presampled_pairs_path, plot_statistics=False,
                                p_copy=0.0, p_sort_by_l2_distance=0.0, presampled_path=presampled_eval_sequences_path, mode="test", sort_criteria="trajectory_shape")
    model_wrapped.set_conditioning_dataset(demonstration_dataset)
    dummy_camera = raw_dataset[0][1]["camera"]
    evaluator = Evaluator(model_wrapped.enc_model, dummy_camera)
else:
    dummy_camera = None
    evaluator = None

text


In [19]:
import random
from mani_skill.examples.cvla.run_env import Args, iterate_env

parsed_args = Args()
parsed_args.env_id = "CvlaMove-v1"
parsed_args.render_mode = "rgb_array"
parsed_args.control_mode = "pd_joint_pos"
parsed_args.camera_views = "random_side"
if conditioning == "trajectory":
    if "cvla-clevr-8" in str(model_path):           # models trained on  clevr-8 should be evaluated on clevr-8
        parsed_args.camera_views = "random_side"
    else:
        parsed_args.camera_views = "random_fov"
parsed_args.quiet = True
# parsed_args.verbose = True
parsed_args.action_encoder = model_wrapped.enc_model.NAME
parsed_args.object_dataset = "objaverse"
#parsed_args.camera_views = "random_fov"
parsed_args.camera_views = "random_side"
parsed_args.obs_mode="rgb+segmentation"

inital_seed = 2919129908
random.seed(inital_seed)
np.random.seed(inital_seed)
torch.manual_seed(inital_seed)
# seeds = random.sample(range(0, 2**32), 50)
# seeds = [2483769646, 180881011, 1809414673, 2883964906, 2170643191, 410500918, 113132070, 3039723796, 14679104, 4279916714, 278991674, 2068020916, 1390357157, 136660899, 3389664825, 2122880614, 1754234965, 3251517866, 1804115595, 1896735568, 412577358, 3372035194, 3506966281, 1703612206, 1768282717, 163537772, 1757877314, 3983804540, 898228096, 2136500906, 2893004833, 1768225131, 1911062959, 2485475605, 2098807074, 1885505790, 3805662650, 3042381630, 2425464850, 2880204856, 322760020, 3904393488, 158321225, 2365191068, 3074121908, 3253628556, 1867392794, 3730714662, 1819615428, 1411403658]
#seeds = [3680886782, 2802635797, 1744512938, 906024569, 1774913893, 2377609938, 2521992606, 172158870, 3004869866, 4226300858, 461653925, 4272151674, 3431745312, 4163708684, 433261562, 885494537, 1098313621, 3217143194, 1031015365, 3395575244, 611105443, 1354071940, 3526388904, 2733645272, 4205874205, 637479328, 1545077070, 2986609316, 658166141, 3537072223, 2232379151, 2612341839, 3437475515, 2554168161, 2177061165, 2160980320, 621132837, 2114964714, 1214631459, 2807144221, 3951810957, 2673307057, 4089274424, 3191539656, 2795800822, 2430345196, 2766575575, 3315258658, 3535103128, 1546654198]
#seeds= seeds[::-1]
# seeds_which_we_wanted_to_use = [2603274749, 915639755, 906024569, 1774913893, 3507534935, 2377609938, 1172049423, 2521992606, 2908100739, 735227406, 172158870, 3646343934, 1088293768, 3004869866, 461653925, 2025933929, 4067874083, 4272151674, 2804080466, 3431745312, 78493986, 3264936548, 4163708684, 433261562, 3808171021, 2068926005, 192656676, 1277534805, 3789057177, 2311723683, 733087580, 4033777556, 2756919050, 3630234651, 3217143194, 1031015365, 3395575244, 2182389079, 2023326672, 79260630, 1354071940, 874206699, 1654640856, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 3536953574, 80174443]
# seeds_clevr_easy = [2603274749, 915639755, 906024569, 1774913893, 3507534935, 2377609938, 1172049423, 2521992606, 2908100739, 735227406, 172158870, 3646343934, 1088293768, 3004869866, 461653925, 2025933929, 4067874083, 4272151674, 2804080466, 3431745312, 78493986, 3264936548, 4163708684, 433261562, 3808171021, 2068926005, 192656676, 1277534805, 3789057177, 2311723683, 733087580, 4033777556, 2756919050, 3630234651, 3217143194, 1031015365, 3395575244, 2182389079, 2023326672, 79260630, 1354071940, 874206699, 1654640856, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 3536953574, 80174443]
# seeds_clevr_hard = [3680886782, 2802635797, 1744512938, 906024569, 1774913893, 2377609938, 2521992606, 172158870, 3004869866, 4226300858, 461653925, 4272151674, 3431745312, 4163708684, 433261562, 885494537, 1098313621, 3217143194, 1031015365, 3395575244, 611105443, 1354071940, 3526388904, 2733645272, 4205874205, 637479328, 1545077070, 2986609316, 658166141, 3537072223, 2232379151, 2612341839, 3437475515, 2554168161, 2177061165, 2160980320, 621132837, 2114964714, 1214631459, 2807144221, 3951810957, 2673307057, 4089274424, 3191539656, 2795800822, 2430345196, 2766575575, 3315258658, 3535103128, 1546654198]
# seeds_objaverse_hard = [1744512938, 2040171868, 603086955, 3114880260, 3453814866, 172158870, 3019050896, 2378207958, 2025933929, 1813581491, 4188321457, 2280014332, 1657617285, 3431745312, 529160525, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 2866478656, 1873976920, 3630234651, 3395575244, 3259409938, 80174443, 4087160184, 2764883367, 3526388904, 1295418175, 517974533, 2532402590, 637479328, 1545077070, 1952321516, 2587484500, 3509369673, 2232379151, 2271163404, 3437475515, 1557090118, 1698846319, 2160980320, 1405630495, 3903676068, 880379348, 1892007745, 2807144221, 4089274424, 86050299]
# seeds_objaverse_easy = [915639755, 2802635797, 1744512938, 2040171868, 1426307309, 1774913893, 1136780770, 3507534935, 4122684191, 3114880260, 1172049423, 3271910715, 3453814866, 735227406, 172158870, 746503233, 1088293768, 3019050896, 2378207958, 4226300858, 3868130567, 1813581491, 4067874083, 4188321457, 4272151674, 2280014332, 1657617285, 3431745312, 529160525, 3264936548, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 1277534805, 733087580, 4033777556, 2866478656, 3630234651, 3395575244, 79260630, 1354071940, 874206699, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 80174443]

#parsed_args.seed = seeds

seeds = random.sample(range(0, 2**32), 1000)
parsed_args.seed = seeds
env_iter = iterate_env(parsed_args, vis=False, model=model_wrapped)

# print all args
print(parsed_args)

Args(env_id='CvlaMove-v1', obs_mode='rgb+segmentation', sim_backend='auto', reward_mode=None, num_envs=1, control_mode='pd_joint_pos', render_mode='rgb_array', shader='default', record_dir=None, pause=False, quiet=True, seed=[2603274749, 753606222, 24081061, 2040171868, 1774913893, 2908100739, 172158870, 461653925, 2280014332, 3431745312, 433261562, 2068926005, 1277534805, 2311723683, 4033777556, 3882745431, 2023326672, 1354071940, 1654640856, 80174443, 4087160184, 927502207, 3835755247, 4205874205, 1545077070, 2216594104, 3509369673, 3537072223, 848189373, 1405630495, 1554653992, 618190537, 1214631459, 2807144221, 1502638877, 988174366, 3387025290, 532401714, 1682090775, 2391768421, 839839333, 360198657, 1609772523, 2402392275, 1656118576, 1787120399, 3921385533, 2046678796, 3725734550, 1394222184, 417121957, 2634723073, 180881011, 2434988439, 926859777, 2170643191, 1292640760, 2586714922, 4279916714, 2068020916, 3743274572, 829688383, 1350322728, 2122880614, 3251517866, 1896735568, 1

In [20]:
%xmode verbose
iters_reached = 0
num_iters = 10
reward_succes = 0
visualize_success = False
visualized_imgs = ""
good_seeds = []
for i in range(num_iters):
    try:
        output = next(env_iter)
    except (ValueError, AssertionError) as e:
        print(f"Encountered error {e.__class__.__name__}")
        print(e)
        print(f"Skipping iteration {i}")
        continue
    
    generated_depth, generated_image = output[0]
    json_dict = output[1]
    tmp_seed = output[2]
    
    print(f"TRY {i:<2} Prefix: {json_dict['prefix']:<100} -> {json_dict['reward']:.2f}")
    if evaluator:
        evaluator.evaluate(json_dict["prediction"], json_dict["suffix"])
    iters_reached = json_dict["iter_reached"]
    good_seeds.append(json_dict["seed"])
    if json_dict["reward"] > 0.75:
        if visualize_success:
            # print(f"suffix: {json_dict['suffix']} -> prediction: {json_dict['prediction']}")
            width, height, _ = generated_image.shape
            camera = DummyCamera(json_dict["camera_intrinsic"], json_dict["camera_extrinsic"], width, height)
            visualized_imgs += render_example(generated_image.detach().cpu().numpy(), json_dict["suffix"], json_dict["prediction"], camera=camera, 
                                              enc_pred=model_wrapped.enc_model, enc=model_wrapped.enc_model, extra_text=f"TRY: {i} - Reward: {json_dict["reward"]:.2f}",
                                              text=json_dict["prefix"])
        reward_succes += 1
    else:
        if i % 100 == 0:
            width, height, _ = generated_image.shape
            camera = DummyCamera(json_dict["camera_intrinsic"], json_dict["camera_extrinsic"], width, height)
            visualized_imgs += render_example(generated_image.detach().cpu().numpy(), json_dict["suffix"], json_dict["prediction"], camera=camera, 
                                              enc_pred=model_wrapped.enc_model, enc=model_wrapped.enc_model, extra_text=f"TRY: {i} - Reward: {json_dict["reward"]:.2f}",
                                              text=json_dict["prefix"])
    
if evaluator:
    stats = evaluator.report_stats()
    for metric_name, metric_value in stats.items():
        print(f"{metric_name}: {metric_value:.3f}")
    evaluator.reset()

iters_reached += 1  # starts with 0 so we add 1 to it

print(f"\nTotal number of tries {iters_reached} for {num_iters} tries gaves {100*num_iters / iters_reached:.2f}% valid sequence rate.")
print(f"Reward success rate: {100*reward_succes / num_iters:.2f}% for model {model_path} with {conditioning} conditioning.")
print(f"Good seeds: {good_seeds}")

Exception reporting mode: Verbose
text description
move mineral onto cube
action encoder xyzrotvec-cam-512xy
filter visible objects True
text description
move human teeth model onto black gold microchips
text description
move smooth brown cushion onto t-rex toy
Encountered error ValueError
Invalid input type. Check that `images` and/or `text` are valid inputs.
Skipping iteration 0


StopIteration: 

In [ ]:
seeds_clevr_easy_50 = [2603274749, 915639755, 906024569, 1774913893, 3507534935, 2377609938, 1172049423, 2521992606, 2908100739, 735227406, 172158870, 3646343934, 1088293768, 3004869866, 461653925, 2025933929, 4067874083, 4272151674, 2804080466, 3431745312, 78493986, 3264936548, 4163708684, 433261562, 3808171021, 2068926005, 192656676, 1277534805, 3789057177, 2311723683, 733087580, 4033777556, 2756919050, 3630234651, 3217143194, 1031015365, 3395575244, 2182389079, 2023326672, 79260630, 1354071940, 874206699, 1654640856, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 3536953574, 80174443]
seeds_clevr_hard_50 = [3680886782, 2802635797, 1744512938, 906024569, 1774913893, 2377609938, 2521992606, 172158870, 3004869866, 4226300858, 461653925, 4272151674, 3431745312, 4163708684, 433261562, 885494537, 1098313621, 3217143194, 1031015365, 3395575244, 611105443, 1354071940, 3526388904, 2733645272, 4205874205, 637479328, 1545077070, 2986609316, 658166141, 3537072223, 2232379151, 2612341839, 3437475515, 2554168161, 2177061165, 2160980320, 621132837, 2114964714, 1214631459, 2807144221, 3951810957, 2673307057, 4089274424, 3191539656, 2795800822, 2430345196, 2766575575, 3315258658, 3535103128, 1546654198]
seeds_objaverse_hard_50 = [1744512938, 2040171868, 603086955, 3114880260, 3453814866, 172158870, 3019050896, 2378207958, 2025933929, 1813581491, 4188321457, 2280014332, 1657617285, 3431745312, 529160525, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 2866478656, 1873976920, 3630234651, 3395575244, 3259409938, 80174443, 4087160184, 2764883367, 3526388904, 1295418175, 517974533, 2532402590, 637479328, 1545077070, 1952321516, 2587484500, 3509369673, 2232379151, 2271163404, 3437475515, 1557090118, 1698846319, 2160980320, 1405630495, 3903676068, 880379348, 1892007745, 2807144221, 4089274424, 86050299]
seeds_objaverse_easy_50 = [915639755, 2802635797, 1744512938, 2040171868, 1426307309, 1774913893, 1136780770, 3507534935, 4122684191, 3114880260, 1172049423, 3271910715, 3453814866, 735227406, 172158870, 746503233, 1088293768, 3019050896, 2378207958, 4226300858, 3868130567, 1813581491, 4067874083, 4188321457, 4272151674, 2280014332, 1657617285, 3431745312, 529160525, 3264936548, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 1277534805, 733087580, 4033777556, 2866478656, 3630234651, 3395575244, 79260630, 1354071940, 874206699, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 80174443]

seeds_objaverse_easy = [915639755, 2802635797, 1744512938, 2040171868, 1426307309, 1136780770, 3507534935, 4122684191, 3114880260, 1172049423, 3271910715, 3453814866, 735227406, 172158870, 746503233, 1088293768, 3019050896, 4226300858, 3868130567, 1813581491, 4067874083, 4188321457, 4272151674, 2280014332, 1657617285, 3431745312, 529160525, 3264936548, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 1277534805, 733087580, 4033777556, 2866478656, 3395575244, 79260630, 1354071940, 874206699, 1654640856, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 80174443, 4087160184, 1417270689, 2764883367, 3526388904, 91860452, 3835755247, 1295418175, 422291714, 517974533, 2532402590, 4193952058, 1545077070, 2108871309, 2216594104, 1952321516, 2986609316, 2587484500, 3509369673, 658166141, 3537072223, 2232379151, 3437475515, 2177061165, 1698846319, 2160980320, 1134777607, 3903676068, 2114964714, 880379348, 3731157486, 1214631459, 1892007745, 736062039, 2807144221, 3951810957, 4089274424, 2795800822, 86050299, 2766575575, 3270628412, 1977278432, 3315258658, 3364530370, 3273855431, 1502638877, 1519891782, 526520938, 3535103128, 988174366, 4244450492, 3098449777, 3998915133, 1446328196, 2283989491, 3670751890, 1637029413, 3400304208, 73237359, 486526149, 2705111285, 1369024863, 3680322809, 4098629731, 2059077229, 3666185719, 1609772523, 964590279, 4029542333, 2169020228, 84107025, 1656118576, 1630161414, 87454936, 4178058369, 3674077824, 2380472908, 1655574748, 3739753431, 1397264776, 3572061631, 3921385533, 862704524, 1520819697, 2658314375, 3725734550, 1394222184, 1287833270, 3283589344, 2132817302, 3611288770, 180881011, 1490696070, 2434988439, 1124294860, 926859777, 937923689, 2883964906, 410500918, 1292640760, 113132070, 1097810371, 3039723796]

#[915639755, 2802635797, 1744512938, 2040171868, 1426307309, 1774913893, 1136780770, 3507534935, 4122684191, 3114880260, 1172049423, 3271910715, 3453814866, 735227406, 172158870, 746503233, 1088293768, 3019050896, 2378207958, 4226300858, 3868130567, 1813581491, 4067874083, 4188321457, 4272151674, 2280014332, 1657617285, 3431745312, 529160525, 3264936548, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 1277534805, 733087580, 4033777556, 2866478656, 3630234651, 3395575244, 79260630, 1354071940, 874206699, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 80174443, 4087160184, 1417270689, 2764883367, 3526388904, 91860452, 3835755247, 1295418175, 422291714, 517974533, 2532402590, 4193952058, 1545077070, 2108871309, 2216594104, 1952321516, 2986609316, 2587484500, 3509369673, 658166141, 3537072223, 2232379151, 2271163404, 3437475515, 2554168161, 2177061165, 1698846319, 2160980320, 1405630495, 1134777607, 3903676068, 2114964714, 880379348, 3731157486, 1892007745, 736062039, 2807144221, 3951810957, 4089274424, 2795800822, 86050299, 2766575575, 3270628412, 1977278432, 3315258658, 3364530370, 3273855431, 1502638877, 1519891782, 526520938, 3535103128, 988174366, 4244450492, 3098449777, 3998915133, 1446328196, 2077609272, 3670751890, 1637029413, 3400304208, 73237359, 2705111285, 1369024863, 3680322809, 4098629731, 2059077229, 3666185719, 1609772523, 964590279, 4029542333, 2169020228, 84107025, 1656118576, 1630161414, 87454936, 4178058369, 3674077824, 2380472908, 1655574748, 3739753431, 2451528178, 1397264776, 3921385533, 862704524, 1520819697, 2658314375, 3725734550, 1394222184, 875433493, 1287833270, 3283589344, 2132817302, 3611288770, 180881011, 1490696070, 2434988439, 1124294860, 926859777, 937923689, 2883964906, 410500918, 1292640760, 113132070, 1097810371, 3039723796, 3237197866, 14679104, 4279916714, 278991674, 3743274572, 136660899, 829688383, 1350322728, 971425815, 2399375446, 3389664825, 2122880614, 1754234965, 2576220809, 1896735568, 693198184, 412577358, 844106427, 487536884, 990419874, 1768282717, 163537772, 1757877314, 3156937422, 898228096, 3500471796, 2806158791, 1169844202, 2893004833, 1768225131, 2793481514, 2098807074, 1885505790, 3805662650, 1457524479, 2425464850, 2536049440, 2880204856, 4238804537, 322760020, 3904393488, 2451226180, 3282225947, 1660021905, 3253628556, 850642649, 2295934963, 3523074427, 3730714662, 605978479, 1819615428, 2764116070, 622825101, 1411403658, 50538578, 1663762609, 1524371290, 1936927061, 1748234018, 874835594, 3248791007, 3231514637, 3309302466, 2640537904, 3477784267, 599063466, 2140489621, 1310773977, 1800136412, 3453062325, 464532259, 3470177567, 1034831155, 3939266399, 1056519846, 3489624216, 1311184519, 2970390600, 37484697, 4264475535, 938213331, 4207407346, 3893435155, 1397347340, 410027214, 3832959828, 2347055186, 3787783058, 4272934799, 2566267809, 2211002086, 3340556848, 3974484705, 998420743, 3391189070, 2180134041]
seeds_objaverse_hard = [1744512938, 2040171868, 1136780770, 603086955, 3114880260, 172158870, 2378207958, 2025933929, 1813581491, 4188321457, 2280014332, 1657617285, 3431745312, 529160525, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 2595664606, 2866478656, 1873976920, 3395575244, 1654640856, 499702150, 3259409938, 4087160184, 2764883367, 3526388904, 1295418175, 517974533, 2532402590, 637479328, 1545077070, 1952321516, 2587484500, 3509369673, 2232379151, 2554168161, 1557090118, 1698846319, 2160980320, 3903676068, 880379348, 1214631459, 2807144221, 4089274424, 86050299, 2766575575, 3315258658, 3364530370, 3273855431, 1502638877, 1519891782, 526520938, 3535103128, 3670751890, 1637029413, 3400304208, 73237359, 2705111285, 839839333, 878241923, 3680322809, 2059077229, 964590279, 4029542333, 2402392275, 84107025, 87454936, 2380472908, 1655574748, 3739753431, 1915827904, 4155052018, 862704524, 1520819697, 2658314375, 417121957, 3611288770, 180881011, 926859777, 937923689, 1292640760, 14679104, 829688383, 1350322728, 971425815, 2399375446, 2122880614, 1804115595, 412577358, 844106427, 487536884, 1703612206, 3500471796, 729860900, 2893004833, 3455918227, 2098807074, 1885505790, 3805662650, 2425464850, 3904393488, 2451226180, 3282225947, 3253628556, 1819615428, 2764116070, 622825101, 1411403658, 50538578, 1663762609, 4197328586, 1748234018, 3477784267, 599063466, 2140489621, 1310773977, 1800136412, 3453062325, 3489624216, 2970390600, 37484697, 938213331, 3893435155, 1115054979, 3787783058, 175045574, 2566267809, 2211002086, 340446616, 3340556848, 3391189070, 2731370221, 2388186141, 861648689, 589984455, 1406048160, 627497341, 344378314, 642151025, 298620094, 4074982120, 3417147514, 206788791, 2995375502, 2815895720, 3970001269, 2858688218]

#[915639755, 1744512938, 2040171868, 1136780770, 603086955, 3114880260, 3453814866, 172158870, 3019050896, 4226300858, 2025933929, 1813581491, 4188321457, 2280014332, 1657617285, 3431745312, 529160525, 4163708684, 2068926005, 192656676, 321034068, 3271584471, 2595664606, 2866478656, 1873976920, 3395575244, 3259409938, 4087160184, 2764883367, 3526388904, 1295418175, 517974533, 2532402590, 637479328, 1545077070, 1952321516, 2587484500, 3509369673, 2232379151, 3437475515, 1557090118, 1698846319, 2160980320, 3903676068, 880379348, 1214631459, 2807144221, 4089274424, 2795800822, 3315258658, 3364530370, 3273855431, 1502638877, 1519891782, 526520938, 3535103128, 2077609272, 2283989491, 3670751890, 1637029413, 3400304208, 73237359, 486526149, 2705111285, 839839333, 878241923, 3680322809, 2059077229, 964590279, 4029542333, 2402392275, 84107025, 87454936, 2380472908, 1655574748, 3739753431, 2451528178, 1915827904, 4155052018, 862704524, 1520819697, 2658314375, 417121957, 3611288770, 180881011, 926859777, 937923689, 410500918, 1292640760, 14679104, 829688383, 1350322728, 971425815, 2399375446, 2122880614, 1804115595, 844106427, 3506966281, 487536884, 1703612206, 1757877314, 3500471796, 729860900, 2893004833, 2793481514, 3455918227, 2098807074, 1885505790, 3805662650, 2425464850, 3904393488, 158321225, 3282225947, 3253628556, 1819615428, 2764116070, 622825101, 1411403658, 50538578, 1663762609, 4197328586, 1748234018, 3477784267, 599063466, 2140489621, 1310773977, 1800136412, 3453062325, 3939266399, 2970390600, 37484697, 938213331, 3893435155, 1115054979, 3787783058, 175045574, 2566267809, 2211002086, 340446616, 3340556848, 3391189070, 2731370221, 2388186141, 861648689, 589984455, 1406048160, 627497341, 344378314, 642151025, 298620094]
seeds_clevr_easy = [2603274749, 915639755, 906024569, 3507534935, 2377609938, 1172049423, 2521992606, 2908100739, 735227406, 172158870, 3646343934, 1088293768, 3004869866, 461653925, 2025933929, 4067874083, 4272151674, 2280014332, 2804080466, 3431745312, 78493986, 3264936548, 4163708684, 433261562, 3808171021, 2068926005, 192656676, 1277534805, 2311723683, 733087580, 4033777556, 2756919050, 3217143194, 1031015365, 3395575244, 2182389079, 2023326672, 79260630, 1354071940, 874206699, 499702150, 2481944716, 3766393311, 3259409938, 2448613074, 3536953574, 80174443, 4087160184, 3610681911, 3835755247, 4205874205, 637479328, 4193952058, 2108871309, 1952321516, 2986609316, 2587484500, 658166141, 3537072223, 2232379151, 2612341839, 2928003231, 2177061165, 3664137434, 2160980320, 1134777607, 1554653992, 621132837, 618190537, 880379348, 1214631459, 1892007745, 736062039, 2597658926, 2807144221, 3951810957, 2673307057, 4089274424, 3191539656, 2795800822, 2430345196, 2766575575, 3270628412, 3315258658, 3535103128, 988174366, 710305527, 4244450492, 3098449777, 3387025290, 2077609272, 3670751890, 1637029413, 3400304208, 532401714, 486526149, 3301601343, 1281830061, 2705111285, 1682090775, 2391768421, 1369024863, 4098629731, 1609772523, 288253476, 964590279, 2207604080, 4029542333, 2332288562, 4123243836, 2169020228, 2402392275, 84107025, 1656118576, 1630161414, 87454936, 4178058369, 3674077824, 2970799875, 1787120399, 1655574748, 2482215029, 3739753431, 2399188649, 2451528178, 1915827904, 4155052018, 1523331802, 1094584044, 1907217244, 3921385533, 3725734550, 1918521600, 1394222184, 2483769646, 4232537362, 180881011, 1809414673, 2883964906, 2170643191, 410500918, 113132070, 3039723796, 14679104, 4279916714, 278991674, 2068020916, 1390357157, 136660899, 3389664825]

seeds_clevr_hard = [3680886782, 2802635797, 1744512938, 906024569, 2377609938, 2521992606, 3453814866, 172158870, 3004869866, 4226300858, 461653925, 4272151674, 2280014332, 3431745312, 4163708684, 433261562, 885494537, 1098313621, 2595664606, 2311723683, 1031015365, 3395575244, 611105443, 1354071940, 80174443, 3526388904, 2733645272, 4205874205, 637479328, 1545077070, 2986609316, 658166141, 3537072223, 2232379151, 2612341839, 2554168161, 2177061165, 2160980320, 621132837, 2114964714, 1214631459, 1892007745, 2807144221, 1370495791, 3951810957, 2673307057, 4089274424, 3191539656, 2795800822, 2430345196, 2766575575, 3315258658, 1502638877, 3535103128, 1546654198, 4244450492, 3098449777, 3670751890, 1637029413, 3400304208, 532401714, 486526149, 3301601343, 2705111285, 1682090775, 288253476, 2332288562, 4123243836, 84107025, 1656118576, 1630161414, 4178058369, 3674077824, 1655574748, 2399188649, 2451528178, 4155052018, 1523331802, 1094584044, 3921385533, 1520819697, 2658314375, 2046678796, 3725734550, 1394222184, 2634723073, 2483769646, 180881011, 2883964906, 113132070, 14679104, 1754234965, 3251517866, 1804115595, 412577358, 1757877314, 1169844202, 2893004833, 1768225131, 3455918227, 1911062959, 2485475605, 1885505790, 2425464850, 2365191068, 3730714662, 4197328586, 1524371290, 1748234018, 874835594, 1000505193, 3231514637, 3309302466, 599063466, 1229900934, 1310773977, 3453062325, 3565481548, 3939266399, 3489624216, 2625986802, 2970390600, 938213331, 2542170577, 2347055186, 2175238691, 175045574, 1342110295, 4192760400, 2566267809, 2211002086, 1521539787, 1268137616, 340446616, 3974484705, 3391189070, 2731370221, 5077479, 1783720456, 1764416308, 861648689, 1861295534, 2878830986, 2763889772, 344378314, 4074982120, 3148680913, 3417147514, 4127206474, 206788791]


if parsed_args.camera_views == "random_fov" and dataset_for_simulation == "objaverse":
    seeds_to_eval = seeds_objaverse_hard
    print("Checking Objaverse hard")
elif parsed_args.camera_views == "random_side" and dataset_for_simulation == "objaverse":
    seeds_to_eval = seeds_objaverse_easy
    print("Checking Objaverse easy")
elif parsed_args.camera_views == "random_fov" and dataset_for_simulation != "objaverse":
    seeds_to_eval = seeds_clevr_hard
    print("Checking CLEVR hard")
elif parsed_args.camera_views == "random_side" and dataset_for_simulation != "objaverse":
    seeds_to_eval = seeds_clevr_easy
    print("Checking CLEVR easy")
else:
    raise ValueError("Unknown camera view or dataset for simulation.")


for g_seed in good_seeds:
    if g_seed not in seeds_to_eval:
        print(f"Seed {g_seed} is NOT in the hard set.")

for orig_seeds in seeds_to_eval:
    if orig_seeds not in good_seeds:
        print(f"Seed {orig_seeds} is NOT in the good seeds.")


In [ ]:
display(HTML(visualized_imgs))